In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
from ultralytics.utils.patches import imread
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np
from PIL import Image, ImageOps
import torch

In [3]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
	"path" : str(DATASETS_DIR.absolute()),
	'train': str(TRAIN_IMAGES_DIR.absolute()),
	'val': str(VAL_IMAGES_DIR.absolute()),
	'test': str(TEST_IMAGES_DIR.absolute()),
	'nc': num_classes,
	'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [4]:
val_df['Image_ID'].nunique()

277

In [5]:
from glob import glob

latest_run_dir = sorted(glob("zindi_challenge_cacao_stage2/train*"), key=lambda x: int(x.split('train')[-1]))[-1]

# Validate the model on the validation set
BEST_PATH = f"{latest_run_dir}/weights/best.pt"
# BEST_PATH = 'zindi_challenge_cacao_stage2/train10/weights/best.pt'
BEST_PATH

'zindi_challenge_cacao_stage2/train10/weights/best.pt'

In [6]:
# Validate the model on the validation set
BEST_CFG = f"{latest_run_dir}/args.yaml"
# BEST_CFG = 'zindi_challenge_cacao_stage2/train8/args.yaml'
BEST_CFG

'zindi_challenge_cacao_stage2/train10/args.yaml'

In [7]:
# Load the trained YOLO model
model = YOLO(BEST_PATH, task="detect", verbose=True).eval()
model.fuse()

YOLO11m summary (fused): 125 layers, 20,032,345 parameters, 0 gradients, 67.7 GFLOPs


In [ ]:
from concurrent.futures import ThreadPoolExecutor

def load_image_(filepath):
	image = Image.open(filepath)
	# return image
	try:
		return ImageOps.exif_transpose(image)
	except Exception:
		pass
	return image


def load_image(filepath):
	return load_image_(filepath)
	return imread(filepath, cv2.IMREAD_COLOR)

def load_images(filepaths):
	with ThreadPoolExecutor() as executor:
		images = list(executor.map(load_image, filepaths))
	return images

In [9]:
with open(BEST_CFG, 'r') as f:
	cfg: dict = yaml.safe_load(f)

In [10]:
# Batch size for predictions
batch_size = 16

cfg["device"] = "cuda"
cfg["batch"] = batch_size
cfg["conf"] = 0.0
cfg["verbose"] = False
cfg["nms"] = True
# cfg["max_det"] = 30
cfg["deterministic"] = True
cfg["iou"] = .6
cfg["agnostic_nms"] = False

cfg.pop("source", None)
# cfg.pop("batch_size")
cfg.pop("visualize", None)
cfg.pop("data", None)
# cfg.pop("name", None)
# cfg.pop("half", None)

cfg.pop("model", None)
cfg["mode"] = "predict"

keys = list(cfg.keys())
for col in keys:
    if (
        "show" in col  # Existing: removes show, show_labels, show_conf, show_boxes
        or "save" in col  # Existing: removes save, save_period, save_json, save_frames, save_txt, save_conf, save_crop, save_dir
        or "freeze" in col  # Existing
        # Consider `col == 'nms'` instead of `"nms" in col` to avoid removing `agnostic_nms`
        # `agnostic_nms` is often useful for prediction.
        # or col == 'nms' # Removes the general nms flag if present
        # or "multi_scale" in col  # Existing
        or "plot" in col  # Existing
        # or "aug" in col  # Existing: removes augment, auto_augment. Also consider removing individual aug params if TTA is off.
        or "drop" in col  # Existing
        # or "iou" in col  # Existing: removes training iou. Prediction uses its own iou parameter.
        or "lr" in col  # Existing: removes lr0, lrf, cos_lr, warmup_bias_lr
        or "mom" in col  # Existing: removes momentum, warmup_momentum
        or "wei" in col  # Existing: removes weight_decay
        # The 'half' parameter is crucial for mixed-precision inference.
        # If cfg['half'] is intended for prediction, this condition should not remove it.
        # or "half" in col # Existing: Problematic if 'half' is needed for prediction.
        # or "nbs" in col  # Existing
        # New conditions:
        or "epoch" in col  # Removes epochs, warmup_epochs
        or col == 'optimizer'
        # or "worker" in col  # Removes workers
        # or col == 'val' or col == 'split' # Removes validation config from training
        or col == 'project' # Removes experiment project name
        or col in ['box', 'cls', 'dfl', 'pose', 'kobj']  # Removes loss component weights
        or col in ['format', 'keras', 'simplify', 'opset', 'int8', 'dynamic', 'workspace'] # Removes export-related params
        or col == 'patience'
        # or col == 'cache'
        # or col == 'seed'
        or col == 'rect' # Rectangular training
        or col == 'resume'
        # or col == 'amp' # Training AMP flag (prediction uses 'half')
        # or col == 'profile'
        # or col == 'tracker'
        # or col == 'task'
        # or col == 'mode' # e.g., mode: train
        or col == 'pretrained'
        # or col == 'deterministic'
        or col == 'exist_ok'
        # or col == 'single_cls'
        or col == 'time' # training time limit
        or col == 'cfg' # path to model cfg yaml (e.g., yolov8n.yaml)
        # If 'augment' key is removed (disabling Test Time Augmentation),
        # you might also want to remove individual augmentation parameters:
        # or col in ['degrees', 'translate', 'scale', 'shear', 'perspective', 'flipud', 'fliplr', 'bgr', 'mosaic', 'mixup', 'cutmix', 'copy_paste', 'erasing']
        # or col.startswith('hsv_') # hsv_h, hsv_s, hsv_v
        # or col == "conf"
        or col == "overlap_mask"
    ):
        cfg.pop(col)

print(cfg)

{'task': 'detect', 'mode': 'predict', 'batch': 16, 'imgsz': 1024, 'cache': False, 'device': 'cuda', 'workers': 4, 'name': 'train10', 'verbose': False, 'seed': 0, 'deterministic': True, 'single_cls': False, 'close_mosaic': 10, 'amp': True, 'fraction': 1.0, 'profile': False, 'multi_scale': True, 'mask_ratio': 4, 'val': True, 'split': 'val', 'conf': 0.0, 'iou': 0.6, 'max_det': 150, 'half': False, 'dnn': False, 'vid_stride': 1, 'stream_buffer': False, 'augment': True, 'agnostic_nms': False, 'classes': None, 'retina_masks': False, 'embed': None, 'line_width': None, 'optimize': False, 'nms': True, 'nbs': 64, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.3, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.1, 'cutmix': 0.0, 'copy_paste': 0.1, 'copy_paste_mode': 'mixup', 'auto_augment': 'augmix', 'erasing': 0.4, 'tracker': 'botsort.yaml'}


In [ ]:
# Path to the test images directory
test_dir_path = VAL_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = [i for i in test_dir_path.glob("*.*") if not i.suffix != ".npy"]

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []


with torch.no_grad():
    # Process images in batches
    for i in tqdm(range(0, len(image_files), batch_size)):
        batch_files = image_files[i : i + batch_size]
        batch_paths = [
            os.path.join(test_dir_path, img_file) for img_file in batch_files
        ]
        batch_images = load_images(
            batch_paths
        )  # [load_image(os.path.join(test_dir_path, img_file)) for img_file in batch_files]

        # Make predictions on the batch of images
        results = model.predict(
            batch_images,
            **cfg,
        )

        # Iterate through each result in the batch
        for img_file, result in zip(batch_files, results):
            boxes = (
                result.boxes.xyxy.tolist() if result.boxes else []
            )  # Bounding boxes in xyxy format
            classes = result.boxes.cls.tolist() if result.boxes else []  # Class indices
            confidences = (
                result.boxes.conf.tolist() if result.boxes else []
            )  # Confidence scores
            names = result.names  # Class names dictionary

            if boxes:  # If detections are found
                for box, cls, conf in zip(boxes, classes, confidences):
                    x1, y1, x2, y2 = box
                    detected_class = names[
                        int(cls)
                    ]  # Get the class name from the names dictionary

                    # Add the result to the all_data list
                    all_data.append(
                        {
                            "Image_ID": str(img_file),
                            "class": detected_class,
                            "confidence": conf,
                            "ymin": y1,
                            "xmin": x1,
                            "ymax": y2,
                            "xmax": x2,
                        }
                    )
            else:  # If no objects are detected
                all_data.append(
                    {
                        "Image_ID": str(img_file),
                        "class": "None",
                        "confidence": None,
                        "ymin": None,
                        "xmin": None,
                        "ymax": None,
                        "xmax": None,
                    }
                )

100%|██████████| 18/18 [00:40<00:00,  2.27s/it]


In [12]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [13]:
sub.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_MYSxE2.jpg,healthy,0.866318,11.774334,400.887085,3264.000000,1906.404175
1,ID_MYSxE2.jpg,healthy,0.179858,0.038642,1627.837402,434.934692,2448.000000
2,ID_MYSxE2.jpg,healthy,0.119070,0.000000,1309.041748,564.720886,1848.757935
3,ID_MYSxE2.jpg,healthy,0.028513,0.000000,1289.234863,568.455994,2448.000000
4,ID_MYSxE2.jpg,healthy,0.018799,1.013141,3.809086,176.854263,452.563324


In [14]:
sub.describe()

,confidence,ymin,xmin,ymax,xmax
count,41550.000000,41550.000000,41550.000000,41550.000000,41550.000000
mean,0.010216,756.784626,705.763058,1213.269992,1211.035150
std,0.077528,1032.883557,816.312349,1197.730495,977.832911
min,0.000005,0.000000,0.000000,0.000000,0.000000
25%,0.000045,0.258480,67.983658,228.984634,429.771835
50%,0.000126,293.386841,424.642853,828.222168,910.262878
75%,0.000460,1172.624329,1031.832306,1853.151825,1788.390900
max,0.945605,4103.251953,4042.481934,4128.000000,4128.000000


In [15]:
sub['class'].value_counts()

class
cssvd          15190
healthy        14641
anthracnose    11719
Name: count, dtype: int64

In [16]:
def load_yolo_labels(label_folder):
	label_data = {}
	label_folder = Path(label_folder)
	paths = [i for i in label_folder.glob("*") if i.suffix != ".npy"]

	for label_file in paths:
		with open(label_file, "r") as file:
			annotations = []
			for line in file:
				parts = line.strip().split()
				if len(parts) == 5:
					class_id, x_center, y_center, width, height = map(float, parts)
					annotations.append({
						"class_id": int(class_id),
						"x_center": x_center,
						"y_center": y_center,
						"width": width,
						"height": height
					})
				else:
					print(f"Skipping line in {label_file}: {line.strip()}")
			label_data[label_file.stem] = annotations
	# Convert the label data to a pandas DataFrame
	label_df = []
	for image_id, annotations in label_data.items():
		for annotation in annotations:
			label_df.append({
				"Image_ID": image_id,
				"class_id": annotation["class_id"],
				"x_center": annotation["x_center"],
				"y_center": annotation["y_center"],
				"width": annotation["width"],
				"height": annotation["height"]
			})

	label_df = pd.DataFrame(label_df)
	return label_df

# Example usage
if os.path.exists('dataset/labels.csv'):
	labels = pd.read_csv('dataset/labels.csv')
else:
	label_folder = VAL_LABELS_DIR
	labels = load_yolo_labels(label_folder)
	labels.to_csv('dataset/labels.csv', index=False)
labels.sample(5)

def yolo_to_bbox(image_folder, labels_df: pd.DataFrame):
	image_folder = Path(image_folder)
	converted_bboxes = []

	paths = [i for i in image_folder.glob("*") if i.suffix != ".npy"]
	for image_file in paths:
		image_id = image_file.stem
		if image_id not in labels_df['Image_ID'].values:
			converted_bboxes.append({
				"Image_ID": image_id,
				"class_id": -1,  # Indicating no label
				"xmin": None,
				"ymin": None,
				"xmax": None,
				"ymax": None
			})

	for _, row in labels_df.iterrows():
		all_ids = [i for i in image_folder.glob(f"{row['Image_ID']}*") if i.suffix != ".npy"]
		image_path = image_folder / f"{row['Image_ID']}"
		if all_ids:
			image_path = all_ids[0]

		if image_path.exists():
			img = load_image_(image_path)
			img_width, img_height = img.size

			x_center = row['x_center'] * img_width
			y_center = row['y_center'] * img_height
			width = row['width'] * img_width
			height = row['height'] * img_height

			x_min = x_center - (width / 2)
			y_min = y_center - (height / 2)
			x_max = x_center + (width / 2)
			y_max = y_center + (height / 2)

			converted_bboxes.append({
				"Image_ID": row['Image_ID'],
				"class_id": row['class_id'],
				"xmin": x_min,
				"ymin": y_min,
				"xmax": x_max,
				"ymax": y_max
			})
		else:
			print(f"Image {image_path} not found.")

	return pd.DataFrame(converted_bboxes)


# Example usage
if os.path.exists('dataset/converted_labels.csv'):
	converted_labels = pd.read_csv('dataset/converted_labels.csv')
else:
	converted_labels = yolo_to_bbox(VAL_IMAGES_DIR, labels)
	converted_labels.to_csv('dataset/converted_labels.csv', index=False)
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
231,ID_TOxurW,0,271.001808,260.001504,2560.000464,3722.001696,anthracnose
330,ID_J5f55n,1,10.000128,1534.000128,648.000768,2021.999616,cssvd
401,ID_xk5CG4,1,262.999980,13.000095,821.000340,807.999705,cssvd
310,ID_GbWjCM,2,536.999400,2485.000704,1082.998872,3264.000000,healthy
140,ID_jMAMzu,0,94.000320,229.999360,621.000000,1152.999680,anthracnose


In [17]:
converted_labels.describe()

,class_id,xmin,ymin,xmax,ymax
count,498.000000,498.000000,498.000000,498.000000,498.000000
mean,1.220884,416.054207,401.291140,1388.212852,1706.086292
std,0.781711,619.027958,556.362058,1021.849436,1231.547325
min,0.000000,-0.002016,-0.002016,56.000256,53.000064
25%,1.000000,7.000240,24.000000,574.999769,727.249294
50%,1.000000,158.000084,156.000192,1075.000032,1280.000000
75%,2.000000,539.250336,566.000482,2180.998764,2717.750610
max,2.000000,3193.998720,3591.000000,4128.000000,4128.002064


In [18]:
converted_labels["Image_ID"].nunique()

277

In [19]:
converted_labels['class_id'].value_counts()

class_id
2    219
1    170
0    109
Name: count, dtype: int64

In [20]:
class_map

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [21]:
id_class_map = {v: k for k, v in class_map.items()}
converted_labels['class'] = converted_labels['class_id'].map(id_class_map)
converted_labels['class'].value_counts()

class
healthy        219
cssvd          170
anthracnose    109
Name: count, dtype: int64

In [22]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
373,ID_FHDhzz,1,0.001224,1986.999168,663.000408,3242.999424,cssvd
124,ID_XXNoaV,2,611.998632,95.999904,2450.998872,3936.000096,healthy
394,ID_dSCbzP,0,28.000320,0.000640,741.000000,1278.000000,anthracnose
98,ID_vDdYaf,1,17.000189,94.000640,795.000611,1280.000000,cssvd
467,ID_a8ILTq,2,886.999752,1653.999360,1508.999832,2368.998144,healthy


In [23]:
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
14630,ID_TSkjv2.jpg,cssvd,0.000044,810.000000,722.338501,810.000000,1080.000000
30765,ID_Nms6Kc.jpeg,anthracnose,0.000230,2863.293213,0.000000,3024.000000,768.516785
20427,ID_WhqRj0.jpeg,anthracnose,0.006055,1870.860352,2381.562500,2358.999268,3218.133057
13473,ID_UrKjY8.jpeg,healthy,0.000092,1620.435425,1564.945068,2419.082275,1785.670166
19071,ID_ET34jY.jpeg,anthracnose,0.000623,696.188049,1267.132446,2827.205566,1785.862549


In [24]:
sub.loc[:, "Image_ID"] = sub["Image_ID"].apply(lambda x: str(Path(x).stem))

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
23692,ID_neqbur,healthy,0.000073,400.487457,205.091171,415.952087,289.436737
33913,ID_MhaDzk,cssvd,0.000844,1.013504,156.206421,245.221573,1270.744873
18527,ID_wIqLck,cssvd,0.000066,191.766968,958.294983,465.139923,1242.323242


In [25]:
def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels["Image_ID"].unique()}

In [26]:
converted_labels.isna().sum()

Image_ID    0
class_id    0
xmin        0
ymin        0
xmax        0
ymax        0
class       0
dtype: int64

In [27]:
ground_truth = convert_df(converted_labels)
ground_truth = {k: ground_truth[k] for k in converted_labels["Image_ID"].unique()}

len(ground_truth)

277

In [28]:
import torch

def calculate_iou_tensor(box1, box2):
	"""
	box1: [4], box2: [4]
	Format: [xmin, ymin, xmax, ymax]
	"""
	xA = torch.max(box1[0], box2[0])
	yA = torch.max(box1[1], box2[1])
	xB = torch.min(box1[2], box2[2])
	yB = torch.min(box1[3], box2[3])

	inter_area = torch.clamp(xB - xA, min=0) * torch.clamp(yB - yA, min=0)
	box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
	box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
	union_area = box1_area + box2_area - inter_area
	return inter_area / union_area if union_area > 0 else torch.tensor(0.0)

def evaluate_detection(predictions, ground_truths, iou_threshold=0.5, conf_threshold=0.0):
    """
    Improved evaluation function with sorted predictions and unique GT matching
    """
    TP = 0
    FP = 0
    FN = 0

    for preds, gts in zip(predictions, ground_truths):
        # Sort predictions by confidence score (descending)
        if len(preds['boxes']) > 0:
            sorted_scores, sorted_indices = torch.sort(preds['scores'], descending=True)
            pred_boxes = preds['boxes'][sorted_indices]
            pred_labels = preds['labels'][sorted_indices]
            pred_scores = preds['scores'][sorted_indices]
        else:
            pred_boxes = torch.empty((0, 4))
            pred_labels = torch.empty((0,), dtype=torch.int32)
            pred_scores = torch.empty((0,))

        gt_boxes = gts['boxes']
        gt_labels = gts['labels']
        matched_gt = set()

        for i in range(len(pred_boxes)):
            if pred_scores[i] < conf_threshold:
                continue
                
            pred_box = pred_boxes[i]
            pred_label = pred_labels[i]
            best_iou = iou_threshold
            best_match = None

            # Find best matching GT box
            for j in range(len(gt_boxes)):
                if j in matched_gt:
                    continue
                if pred_label != gt_labels[j]:
                    continue
                iou = calculate_iou_tensor(pred_box, gt_boxes[j])
                if iou > best_iou:
                    best_iou = iou
                    best_match = j

            if best_match is not None:
                TP += 1
                matched_gt.add(best_match)
            else:
                FP += 1

        FN += len(gt_boxes) - len(matched_gt)

    precision = TP / (TP + FP) if (TP + FP) else 0.0
    recall = TP / (TP + FN) if (TP + FN) else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
    accuracy = TP / (TP + FP + FN) if (TP + FP + FN) else 0.0

    return {
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1_score,
        'Accuracy': accuracy
    }

In [29]:
sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
32075,ID_E2z7VZ,healthy,0.000056,0.000000,0.579109,399.625702,236.093719
4848,ID_P1lHSv,anthracnose,0.000130,0.137482,1216.930664,77.187302,1401.965820
27783,ID_H7BwD8,cssvd,0.000183,1760.022827,84.435204,3543.182617,1172.902466


In [30]:
sub["class_id"] = sub["class"].map(class_map)

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
33811,ID_crD9na,healthy,0.000045,1823.491089,1871.566406,3264.000000,2418.143311,2
24157,ID_C9SxS3,anthracnose,0.024281,1155.439209,1.369324,1455.512939,128.413757,0
21832,ID_A4KXll,cssvd,0.000029,0.000000,366.817780,94.524017,960.000000,1


In [31]:
predictions = get_preds_data(sub, None)

len(predictions)

277

In [32]:
for i in np.linspace(0.0, 0.95, 15):
	scores = evaluate_detection(
		predictions.values(),
		ground_truth.values(),
		iou_threshold=0.5,
		conf_threshold=i
	)
	print("Evaluation metric at:", i, " score :", scores)

Evaluation metric at: 0.0  score : {'TP': 480, 'FP': 41070, 'FN': 18, 'Precision': 0.011552346570397111, 'Recall': 0.963855421686747, 'F1 Score': 0.022831050228310498, 'Accuracy': 0.011547344110854504}
Evaluation metric at: 0.06785714285714285  score : {'TP': 419, 'FP': 362, 'FN': 79, 'Precision': 0.5364916773367477, 'Recall': 0.8413654618473896, 'F1 Score': 0.655199374511337, 'Accuracy': 0.4872093023255814}
Evaluation metric at: 0.1357142857142857  score : {'TP': 402, 'FP': 237, 'FN': 96, 'Precision': 0.6291079812206573, 'Recall': 0.8072289156626506, 'F1 Score': 0.7071240105540897, 'Accuracy': 0.5469387755102041}
Evaluation metric at: 0.20357142857142857  score : {'TP': 393, 'FP': 158, 'FN': 105, 'Precision': 0.7132486388384754, 'Recall': 0.7891566265060241, 'F1 Score': 0.7492850333651095, 'Accuracy': 0.5990853658536586}
Evaluation metric at: 0.2714285714285714  score : {'TP': 381, 'FP': 119, 'FN': 117, 'Precision': 0.762, 'Recall': 0.7650602409638554, 'F1 Score': 0.7635270541082164, 

In [33]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
220,ID_qKxdD2,2,2206.001952,0.000000,3411.001440,3024.000000,healthy
419,ID_lLn5Lv,1,379.999488,1440.000000,1296.999168,2044.000256,cssvd
46,ID_fiFUkh,2,0.001512,68.997600,877.000824,991.999008,healthy
465,ID_a8ILTq,2,520.999272,1505.999808,1251.999000,1930.998720,healthy
312,ID_ka9PpD,1,0.000405,0.000000,724.000275,1080.000000,cssvd


In [34]:
import torch
from torchmetrics.detection import MeanAveragePrecision


def compute_map(preds, targets):
    # Initialize the metric
    metric = MeanAveragePrecision(
        iou_thresholds=[0.5],
        max_detection_thresholds=[1, 10, cfg["max_det"]],
        class_metrics=True,
    )

    # Update metric with predictions and targets
    metric.update(preds, targets)

    # Compute the results
    result = metric.compute()

    return result

preds = list(get_preds_data(sub, 0).values())
targets = list(ground_truth.values())

results = compute_map(preds, targets)
print("mAP Results:", 0, " - ", results)
# thrs = np.linspace(0.000001, 0.95, 15)
# for i in thrs:
#     preds = list(get_preds_data(sub, i).values())

#     targets = list(ground_truth.values())

#     # Compute mAP
#     results = compute_map(preds, targets)

#     # Print results
#     print("mAP Results:", i, " - ", results)

mAP Results: 0  -  {'map': tensor(-1.), 'map_50': tensor(0.8043), 'map_75': tensor(-1.), 'map_small': tensor(-1.), 'map_medium': tensor(0.7707), 'map_large': tensor(0.8073), 'mar_1': tensor(0.5252), 'mar_10': tensor(0.9246), 'mar_150': tensor(0.9665), 'mar_small': tensor(-1.), 'mar_medium': tensor(0.9444), 'mar_large': tensor(0.9669), 'map_per_class': tensor([-1., -1., -1.]), 'mar_150_per_class': tensor([0.9817, 0.9588, 0.9589]), 'classes': tensor([0, 1, 2], dtype=torch.int32)}


In [35]:
sub.to_csv('dataset/evaluations/validation.csv', index=False)

In [36]:
import pandas as pd

sub = pd.read_csv('dataset/evaluations/validation.csv')
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
24398,ID_GkeQ6m,anthracnose,0.000012,0.000000,0.000000,130.657364,335.036194,0
28568,ID_hADR5u,healthy,0.000181,2.835645,491.694336,343.549194,758.576538,2
7347,ID_jj4SlG,healthy,0.000040,316.104004,526.047546,880.039062,875.812805,2
39144,ID_kk7SDW,anthracnose,0.000101,2251.368164,1989.372681,3587.797119,2396.187500,0
39238,ID_J5f55n,cssvd,0.000080,1990.370605,218.577942,2047.932617,626.798767,1


In [37]:
model.overrides

{'task': 'detect',
 'data': 'data.yaml',
 'imgsz': 1024,
 'single_cls': False,
 'model': 'zindi_challenge_cacao_stage2/train10/weights/best.pt'}